In [1]:
import time
import sys
sys.path.append('../')
import librairies.dagfeaturingfx 
from  librairies.dagfeaturingfx import *
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm
import joblib
from joblib import Parallel,delayed
import pyttsx3
from sklearn.preprocessing import MinMaxScaler,MaxAbsScaler,quantile_transform,PolynomialFeatures
from librairies.bt import *
from sklearn.metrics import accuracy_score, make_scorer, precision_score, recall_score, precision_recall_curve, confusion_matrix, classification_report
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from numpy import sqrt
from numpy import argmax
import colorama as col
from collections import Counter
from librairies.strategy import *
scaler = MinMaxScaler(feature_range=(0, 1))
engine = pyttsx3.init()
engine.say("Librairies loaded")
engine.runAndWait() 

Importing Librairies...


In [ ]:
###### RELOAD LIBRAIRIE #####
from importlib import reload
sys.path.append('../')
librairies.dagfeaturingfx= reload(librairies.dagfeaturingfx)
from  librairies.dagfeaturingfx import *

%%time

_t1 = dt.datetime.now()
print('Début des opérations horodatée à',col.Fore.YELLOW,dt.datetime.now(),col.Style.RESET_ALL)

TICKER_LIST = ['EUR/USD']
x=TICKER_LIST[0]
_ticker = x.replace('/','')
_period = 'm5'
df_all = pd.read_csv(x.replace('/','')+'_'+_period+'_BidAndAsk.csv')

##### Ajout de la colonne Symbol pour identifier le ticker
df_all['Symbol'] = _ticker

##### On fixe la date en index sous forme de Timestamp
df_all['Lindex'] = pd.to_datetime(df_all['Date'] + ' ' + df_all['Time'])
df_all.set_index(pd.to_datetime(df_all.Lindex,format='%Y-%m-%d %H:%M:%S'),drop=True,inplace=True)

###### On drop les colonnes inutiles
df_all = df_all.drop(['Date','Lindex','Time','Total Ticks'],axis=1)

##### On enlève les jours correspondant au samedi et au dimanche
df_all['WE'] = np.where(((df_all.index.weekday == 5) | (df_all.index.weekday == 6)),None,df_all.index.weekday)
df_all = df_all.dropna()
df_all = df_all.drop(['WE'],axis=1)

##### Calcul des averages pour les OHLC
df_all['Open'] = (df_all['OpenBid'] + df_all['OpenAsk']) / 2
df_all['High'] = (df_all['HighBid'] + df_all['HighAsk']) / 2
df_all['Low'] = (df_all['LowBid'] + df_all['LowAsk']) / 2
df_all['Close'] = (df_all['CloseBid'] + df_all['CloseAsk']) / 2

hourly_all = pd.read_csv(x.replace('/','')+'_H1_BidAndAsk.csv')

##### Ajout de la colonne Symbol pour identifier le ticker
hourly_all['Symbol'] = _ticker

##### On fixe la date en index sous forme de Timestamp
hourly_all['Lindex'] = pd.to_datetime(hourly_all['Date'] + ' ' + hourly_all['Time'])
hourly_all.set_index(pd.to_datetime(hourly_all.Lindex,format='%Y-%m-%d %H:%M:%S'),drop=True,inplace=True)

###### On drop les colonnes inutiles
hourly_all = hourly_all.drop(['Date','Lindex','Time','Total Ticks'],axis=1)

##### On enlève les jours correspondant au samedi et au dimanche
hourly_all['WE'] = np.where(((hourly_all.index.weekday == 5) | (hourly_all.index.weekday == 6)),None,hourly_all.index.weekday)
hourly_all = hourly_all.dropna()
hourly_all = hourly_all.drop(['WE'],axis=1)

##### Calcul des averages pour les OHLC
hourly_all['Open'] = (hourly_all['OpenBid'] + hourly_all['OpenAsk']) / 2
hourly_all['High'] = (hourly_all['HighBid'] + hourly_all['HighAsk']) / 2
hourly_all['Low'] = (hourly_all['LowBid'] + hourly_all['LowAsk']) / 2
hourly_all['Close'] = (hourly_all['CloseBid'] + hourly_all['CloseAsk']) / 2

engine.say("Processing featuring of dataframes, daily and intra-day")
engine.runAndWait()
##### Récupération des data pour tous les tickers sur la période demandée en intraday => df_all

engine.say("Raw data are loaded in memory")
engine.runAndWait()


df_all = timerange1D(df_all)
hourly_all = timerange1D(hourly_all)
daily_all = get_daily(hourly_all,TICKER_LIST)

engine.say("Daily up to date")
engine.runAndWait()

daily_all = timerange1W(daily_all)
weekly_all = get_weekly(daily_all,TICKER_LIST)
engine.say("Weekly up to date")
engine.runAndWait()

##### On calcule l'ADR sur le daily
daily_all = adr(daily_all,_window=14)
engine.say("ADR computed")
engine.runAndWait()

##### On récupère l'ADR qui a été calculé en daily (daily_all) pour le mettre dans la base intraday df_all
df_all = getadr(daily_all,df_all,TICKER_LIST)
engine.say("ADR get in daily")
engine.runAndWait()

df_all = adrhnl(daily_all,df_all,TICKER_LIST)

##### Calcul d'une SMA 200 sur df_all
df_all = sma(df_all=df_all,_window=200)

##### Calcul des bollinger sur df_all
df_all = bollinger(df_all,_slow=20)

##### Calcul du stochastic slow. Par défaut les paramètres sont 5 et 3.
df_all = slowstochastic(df_all,TICKER_LIST)

df_all = ema(df_all,21,TICKER_LIST)

df_all = ema(df_all,8,TICKER_LIST)

weekly_all = pivot(weekly_all,TICKER_LIST)

df_all = pivotimportdf(df_all,weekly_all,TICKER_LIST)

df_all = atr(df_all,TICKER_LIST,14)

df_all = rvi(df_all,TICKER_LIST,_window=14)

df_all = sbgamma(df_all,TICKER_LIST)

df_all = onhisma(df_all,TICKER_LIST,_window=5)
df_all = onlosma(df_all,TICKER_LIST,_window=5)

df_all = onhisma(df_all,TICKER_LIST,_window=21)
df_all = onlosma(df_all,TICKER_LIST,_window=21)

df_all = onhisma(df_all,TICKER_LIST,_window=34)
df_all = onlosma(df_all,TICKER_LIST,_window=34)

df_all = importohlc(df_all,weekly_all,TICKER_LIST,_suffix='_weekly')

df_all = importohlc(df_all=df_all,other_all=daily_all,TICKER_LIST=TICKER_LIST,_suffix='_daily')

engine.say("The job is done")
engine.runAndWait()


engine.say("AAll the bases are saved")
engine.runAndWait()

_t2 = dt.datetime.now()
print('Début des opérations horodatée à',col.Fore.YELLOW,dt.datetime.now(),col.Style.RESET_ALL)
print((_t2 - _t1))

In [3]:
%%time

_t1 = dt.datetime.now()
print('Début des opérations horodatée à',col.Fore.YELLOW,dt.datetime.now(),col.Style.RESET_ALL)

TICKER_LIST = ['EUR/USD']
x=TICKER_LIST[0]
_ticker = x.replace('/','')
_period = 'm5'
df_all = pd.read_csv(x.replace('/','')+'_'+_period+'_BidAndAsk.csv')

##### Ajout de la colonne Symbol pour identifier le ticker
df_all['Symbol'] = _ticker

##### On fixe la date en index sous forme de Timestamp
df_all['Lindex'] = pd.to_datetime(df_all['Date'] + ' ' + df_all['Time'])
df_all.set_index(pd.to_datetime(df_all.Lindex,format='%Y-%m-%d %H:%M:%S'),drop=True,inplace=True)

###### On drop les colonnes inutiles
df_all = df_all.drop(['Date','Lindex','Time','Total Ticks'],axis=1)
##### Calcul des averages pour les OHLC
df_all['Open'] = (df_all['OpenBid'] + df_all['OpenAsk']) / 2
df_all['High'] = (df_all['HighBid'] + df_all['HighAsk']) / 2
df_all['Low'] = (df_all['LowBid'] + df_all['LowAsk']) / 2
df_all['Close'] = (df_all['CloseBid'] + df_all['CloseAsk']) / 2
df_all = timerange1D(df_all)
df_all = slowstochastic(df_all,TICKER_LIST)

Début des opérations horodatée à  2021-03-03 22:34:21.233197 

Ajout Date
100%|██████████| 1/1 [00:00<00:00, 2824.45it/s]
Calcul du Slow STOCHASTIC
Fenêtre de Glissement : 5 periodes.
Période de lissage : 3 periodes.

CPU times: user 1min 3s, sys: 2.39 s, total: 1min 5s
Wall time: 1min 7s


In [7]:
%%time
joblib.dump(df_all,x.replace('/','')+'_DF_ALL')


CPU times: user 1.18 s, sys: 307 ms, total: 1.49 s
Wall time: 1.69 s


['EURUSD_DF_ALL']

In [71]:
%%time
TICKER_LIST = ['EUR/USD']
x=TICKER_LIST[0]
df_all = joblib.load(x.replace('/','')+'_DF_ALL')


CPU times: user 287 ms, sys: 107 ms, total: 394 ms
Wall time: 393 ms


In [73]:
%%time
####### CHOIX DE LA TIME FRAME
_period = 'm5'
##### Récupération de la liste des tickers à partir du répertoire où sont rangées les bases
TICKER_LIST = ['EUR/USD']
TIK = ['JPY','CHF','CAD','GBP','AUD','NZD','SEK','NOK','MXN','ZAR','TRY','ILS','CNH','USD','HKD','0']
RATE = [105.4,0.8989,1.276,0.877,0.7676,0.7201,8.3684,8.5238,20.09,14.81,7.03,3.28,6.458,1,7.7527,1.21]
df_ratefx = pd.DataFrame(index=TIK)
df_ratefx['rate'] = RATE
#############################
##### F E A T U R I N G #####
#############################

print(df_all.shape)
# Make our choice for the split, compliant to our backtest 
_start = '2010-01-01' # start the train there '2010-01-01'
_mid = '2018-08-29' # stop the train and begin the test there '2016-08-31'
_stop = '2019-12-30' # stop the test there. After that, it is kept for oos
_last = '2021-02-15' # '2020-12-31'
#### S T R A T E G Y

df_all = stochastic(df_all)
##### On backtest selon le ticker selectionné sur la période déterminée

x=TICKER_LIST[0]
print(x)
_year_bottom = _stop
_year_top = _last
_nb_bougie_exit = 2000000
_trigger_reengage = 0
_trigger_target = 1
_trigger_invers = 0
_trigger_sl = 1
_verbose = 0
_cash_ini = 200000
_rate = 1/df_ratefx.loc[x[4:],'rate']
backtest = df_all[df_all.Symbol == x.replace('/','')]
_target =  0.002
_exposure = 10
_size = 200000
_sl =  0.001
_open_hour = 8 # day only
_close_hour = 23 # day only
_window = 0 # day only
print()
print(col.Fore.RED,'###############################################################################################')
print(' ####################################### OOS WITHOUT AI ########################################')
print(' ###############################################################################################',col.Style.RESET_ALL)
##### Backtest Over Night
TRACKER,_nb_looser = bt(backtest,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)

print()
print(col.Fore.CYAN,'###############################################################################################')
print(' #################################### TRAIN/TEST WITHOUT AI ####################################')
print(' ###############################################################################################',col.Style.RESET_ALL)
_year_bottom = _start
_year_top = _stop

TRACKER,_nb_looser = bt(backtest,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)

print()
print(col.Fore.BLUE,'###############################################################################################')
print(' #################################### DENOISING & ENHANCING ####################################')
print(' ###############################################################################################',col.Style.RESET_ALL)
while _nb_looser > 0 :
    
    df_all['TRACKER'] = np.where(df_all.index.isin(TRACKER),1,0)
    df_all['Valid'] = np.where(((df_all.Signal!=0)&(df_all.TRACKER==1)),1,0)
    df_all['Signal'] = np.where(((df_all.Valid==1)&(df_all.Signal==1)),1,np.where(((df_all.Valid==1)&(df_all.Signal==-1)),-1,0))
    backtest = df_all[df_all.Symbol == x.replace('/','')]
    ##### Purification of signal by denoising and enhancing
    TRACKER,_nb_looser = bt(backtest,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
            _rate,x,_target,_exposure,_size,_sl)

df_all = joblib.load(x.replace('/','')+'_DF_ALL')
df_all = stochastic(df_all)
#features = featuring(df_all)
features = df_all.copy()

features['TRACKER'] = np.where(features.index.isin(TRACKER),1,0)

# First, we must have an output. We'll call it 'Valid'. It wil be where Tracker & Signal are both to 1
features['Valid'] = np.where(((features.Signal!=0)&(features.TRACKER==1)),1,0) # Don't miss the point that even a Signal -1 must be considered as a good one by TRACKER

# Let's isolate the ticker on which we made the test
#features = features[features.Symbol==x.replace('/','')]
#features = df_all.copy()
# And drop the nan
features = features.dropna()
##### Signal is from strategy. This is potential good one. But we have to create the TRACKER column where the Signal where efficient

features['deltaHL'] = features['High'] - features['Low']
features['deltaOC'] = features['Open'] - features['Close']

features = features.dropna()
features = features[features.Symbol==_ticker]
features['TRACKER'] = np.where(features.index.isin(TRACKER),1,0)
features_train = features[(features.Date>=_start)&(features.Date<=_mid)]
features_test = features[(features.Date>_mid)&(features.Date<=_stop)]
features_oos = features[(features.Date>_stop)&(features.Date <= _last)]
features_train = features_train[features_train.Signal!=0]
features_test = features_test[features_test.Signal!=0]


features_train['OpenBid'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.OpenBid.astype(np.float32)).reshape(-1, 1)))
features_train['HighBid'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.HighBid.astype(np.float32)).reshape(-1, 1)))
features_train['LowBid'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.LowBid.astype(np.float32)).reshape(-1, 1)))
features_train['CloseBid'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.CloseBid.astype(np.float32)).reshape(-1, 1)))
features_train['HighAsk'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.HighAsk.astype(np.float32)).reshape(-1, 1)))
features_train['LowAsk'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.LowAsk.astype(np.float32)).reshape(-1, 1)))
features_train['CloseAsk'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.CloseAsk.astype(np.float32)).reshape(-1, 1)))
features_train['Open'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.Open.astype(np.float32)).reshape(-1, 1)))
features_train['High'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.High.astype(np.float32)).reshape(-1, 1)))
features_train['Low'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.Low.astype(np.float32)).reshape(-1, 1)))
features_train['Close'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.Close.astype(np.float32)).reshape(-1, 1)))
features_train['slow_K5'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.slow_K5.astype(np.float32)).reshape(-1, 1)))
features_train['slow_D5'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.slow_D5.astype(np.float32)).reshape(-1, 1)))
features_train['OpenAsk'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.OpenAsk.astype(np.float32)).reshape(-1, 1)))
features_train['deltaHL'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.deltaHL.astype(np.float32)).reshape(-1, 1)))
features_train['deltaOC'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.deltaOC.astype(np.float32)).reshape(-1, 1)))

features_test['OpenBid'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.OpenBid.astype(np.float32)).reshape(-1, 1)))
features_test['HighBid'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.HighBid.astype(np.float32)).reshape(-1, 1)))
features_test['LowBid'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.LowBid.astype(np.float32)).reshape(-1, 1)))
features_test['CloseBid'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.CloseBid.astype(np.float32)).reshape(-1, 1)))
features_test['HighAsk'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.HighAsk.astype(np.float32)).reshape(-1, 1)))
features_test['LowAsk'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.LowAsk.astype(np.float32)).reshape(-1, 1)))
features_test['CloseAsk'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.CloseAsk.astype(np.float32)).reshape(-1, 1)))
features_test['Open'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.Open.astype(np.float32)).reshape(-1, 1)))
features_test['High'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.High.astype(np.float32)).reshape(-1, 1)))
features_test['Low'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.Low.astype(np.float32)).reshape(-1, 1)))
features_test['Close'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.Close.astype(np.float32)).reshape(-1, 1)))
features_test['slow_K5'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.slow_K5.astype(np.float32)).reshape(-1, 1)))
features_test['slow_D5'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.slow_D5.astype(np.float32)).reshape(-1, 1)))
features_test['OpenAsk'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.OpenAsk.astype(np.float32)).reshape(-1, 1)))
features_test['deltaHL'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.deltaHL.astype(np.float32)).reshape(-1, 1)))
features_test['deltaOC'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.deltaOC.astype(np.float32)).reshape(-1, 1)))


features_oos['OpenBid'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.OpenBid.astype(np.float32)).reshape(-1, 1)))
features_oos['HighBid'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.HighBid.astype(np.float32)).reshape(-1, 1)))
features_oos['LowBid'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.LowBid.astype(np.float32)).reshape(-1, 1)))
features_oos['CloseBid'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.CloseBid.astype(np.float32)).reshape(-1, 1)))
features_oos['HighAsk'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.HighAsk.astype(np.float32)).reshape(-1, 1)))
features_oos['LowAsk'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.LowAsk.astype(np.float32)).reshape(-1, 1)))
features_oos['CloseAsk'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.CloseAsk.astype(np.float32)).reshape(-1, 1)))
features_oos['Open'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.Open.astype(np.float32)).reshape(-1, 1)))
features_oos['High'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.High.astype(np.float32)).reshape(-1, 1)))
features_oos['Low'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.Low.astype(np.float32)).reshape(-1, 1)))
features_oos['Close'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.Low.astype(np.float32)).reshape(-1, 1)))
features_oos['slow_K5'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.slow_K5.astype(np.float32)).reshape(-1, 1)))
features_oos['slow_D5'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.slow_D5.astype(np.float32)).reshape(-1, 1)))
features_oos['OpenAsk'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.OpenAsk.astype(np.float32)).reshape(-1, 1)))
features_oos['deltaOC'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.deltaOC.astype(np.float32)).reshape(-1, 1)))
features_oos['deltaOC'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.deltaOC.astype(np.float32)).reshape(-1, 1)))

# Proceed an MaxAbsScaler on features
#features_train,features_test,features_oos = scaling(features,x.replace('/',''),TRACKER,_start,_mid,_stop,_last,scaler=MaxAbsScaler())
#features_train,features_test,features_oos = quantile(features_train,features_test,features_oos,quantile_transform)

from sklearn.tree import DecisionTreeClassifier
_name = 'DecisionTreeClassifier'
#_model = MLPClassifier(hidden_layer_sizes=(300,15), activation='relu', solver='adam', alpha=0.000001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=400, shuffle=True, random_state=26, tol=0.0000001, verbose=False, warm_start=True, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.8, beta_2=0.999, epsilon=1e-08)
_model = DecisionTreeClassifier(criterion='entropy',random_state=26,splitter='best',ccp_alpha=0.00007,max_features=7,class_weight={0: 1, 1: 1})

_model.fit(features_train.drop(['Date','Symbol','TRACKER','Valid','Signal'],axis=1),features_train.Valid)
yhat = _model.predict(features_test.drop(['Date','Symbol','TRACKER','Valid','Signal'],axis=1))
accu = round(accuracy_score(features_test.Valid, yhat) * 100,2)
prec = round(precision_score(features_test.Valid, yhat,pos_label=1) * 100,2)
recall = round(recall_score(features_test.Valid, yhat) * 100,2)
f1 = round(f1_score(features_test.Valid, yhat) * 100,2)


print('Signaux - Accuracy :' ,accu,'%')
print('Signaux - Precision :',prec,'%')
print('Signaux - Recall :', recall,'%')
print('Achat - F-measure: :' ,f1,'%')
print('\n')
print(classification_report(features_test.Valid, yhat, target_names=['Neg', 'Pos']))
conf_matrix = pd.DataFrame(columns=['Bonnes_Estimations','Mauvaises_Estimations'])
tt = confusion_matrix(features_test.Valid, yhat, labels=[1,0])    #_model.classes_)

conf_matrix.loc['vrais-réels'] = tt[0]
conf_matrix.loc['faux-réels'] = tt[1]

print(conf_matrix)
print()
print(col.Fore.BLUE,'Signaux pour',col.Fore.YELLOW,x,col.Style.RESET_ALL)
_tp = tt[0][0]
_fn = tt[0][1]
_prec = round((tt[0][0]/(tt[0][0]+tt[0][1]))*100,2)
_rec = round((tt[0][1]/(tt[0][0]+tt[0][1]))*100,2)

print('Vrais signaux trouvés    : ',tt[0][0])
print('Vrais signaux non trouvé :',tt[0][1])
print('Total des vrais signaux  :',tt[0][0]+tt[0][1])
if _prec > 69 and prec > 69 :
    print(col.Fore.GREEN)
elif _prec < 51 or prec < 51 :
    print(col.Fore.RED)
else:
    print(col.Fore.YELLOW)
print('Precision : ',_prec,'%')
if _rec > 69 and _rec > 69 :
    print(col.Fore.GREEN)
elif _rec < 51 or _rec < 51 :
    print(col.Fore.RED)
else:
    print(col.Fore.YELLOW)
print('Recall',_rec,'%')
print(col.Style.RESET_ALL)

df_all_oos = df_all[(df_all.Date > _stop)&(df_all.Date <= _last)].dropna()

df_all_oos = df_all_oos[df_all_oos.Symbol==x.replace('/','')]


df_all_oos['Valid'] = _model.predict(features_oos.drop(['Date','Symbol','Signal','TRACKER','Valid'],axis=1))


df_all_oos['Signal'] = np.where((df_all_oos.Signal==1)&(df_all_oos.Valid==1),1,np.where((df_all_oos.Signal==-1)&(df_all_oos.Valid==1),-1,0))
 
##### On backtest selon le ticker selectionné sur la période déterminée

joblib.dump(_model,'Save_'+x.replace('/','')+'_m5.sav')

print(x)
_year_bottom = _stop
_year_top = _last

_trigger_invers = 1
_target = 0.017
##### Backtest Over Night
FINAL_TRACKER = bt(df_all_oos,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)

(835382, 17)
EUR/USD

 ###############################################################################################
 ####################################### OOS WITHOUT AI ########################################
 ############################################################################################### 
Librairies imported

Début des opérations horodatée à 2021-03-04 10:59:59.468897

Chargement de la nouvelle base


 Le rate du ticker EUR/USD est à  1.0 
Bases chargées
TETEL process effectué
 ENTERING THE BACKTEST 
100%|██████████| 84912/84912 [00:05<00:00, 15807.94it/s]
 For ticker  EUR/USD 
 
Total Profit & Loss : $  -50302.0 . En  1460   transactions.
 
Winners Number : 457 
 
Loosers number : 1003 
BT's execution time 0:00:11.680692
 EUR/USD  results 
 Tested Period 2019-12-30  à 2021-02-15 
 Total Number of trades 1460 
Started Cash : 200000
P&L in currency:  -50302.0$ 
P&L in %:  -25.15% 
Average trade duration 44.59
# Winners  457.0
# Winners long  210.0
# Winners short

In [98]:
%%time
import time
import sys
sys.path.append('../')
import librairies.dagfeaturingfx 
from  librairies.dagfeaturingfx import *
from matplotlib import pyplot as plt
import seaborn as sns
from tqdm import tqdm
import joblib
from joblib import Parallel,delayed
import pyttsx3
from sklearn.preprocessing import MinMaxScaler,MaxAbsScaler,quantile_transform,PolynomialFeatures
from librairies.bt import *
from sklearn.metrics import accuracy_score, make_scorer, precision_score, recall_score, precision_recall_curve, confusion_matrix, classification_report
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from numpy import sqrt
from numpy import argmax
import colorama as col
from collections import Counter
from librairies.strategy import *
scaler = MinMaxScaler(feature_range=(0, 1))

TICKER_LIST = ['EUR/USD']
x=TICKER_LIST[0]
df_all = joblib.load(x.replace('/','')+'_DF_ALL')


_start = '2010-01-01' # start the train there '2010-01-01'
_mid = '2018-08-29' # stop the train and begin the test there '2016-08-31'
_stop = '2019-12-30' # stop the test there. After that, it is kept for oos
_last = '2021-02-15' 

TICKER_LIST = ['EUR/USD']
x=TICKER_LIST[0]
_ticker = x.replace('/','')
TRACKER = joblib.load('TRACKER')
df_all = joblib.load(x.replace('/','')+'_DF_ALL')

df_all = stochastic(df_all)
#features = featuring(df_all)
features = df_all.copy()

features['TRACKER'] = np.where(features.index.isin(TRACKER),1,0)

# First, we must have an output. We'll call it 'Valid'. It wil be where Tracker & Signal are both to 1
features['Valid'] = np.where(((features.Signal!=0)&(features.TRACKER==1)),1,0) # Don't miss the point that even a Signal -1 must be considered as a good one by TRACKER

# Let's isolate the ticker on which we made the test
#features = features[features.Symbol==x.replace('/','')]
#features = df_all.copy()
# And drop the nan
features = features.dropna()
##### Signal is from strategy. This is potential good one. But we have to create the TRACKER column where the Signal where efficient

features['deltaHL'] = features['High'] - features['Low']
features['deltaOC'] = features['Open'] - features['Close']
features['deltaHC'] = features['High'] - features['Close']
features['deltaLC'] = features['Low'] - features['Close']
features['deltaOH'] = features['Open'] - features['High']
features['deltaOL'] = features['Open'] - features['Low']

features = features.drop(['OpenBid','OpenAsk','HighBid','LowBid','CloseBid','HighAsk','LowAsk','CloseAsk','Open','High','Low','Close'],axis=1)

features = features.dropna()
features = features[features.Symbol==_ticker]
features['TRACKER'] = np.where(features.index.isin(TRACKER),1,0)
features_train = features[(features.Date>=_start)&(features.Date<=_mid)]
features_test = features[(features.Date>_mid)&(features.Date<=_stop)]
features_oos = features[(features.Date>_stop)&(features.Date <= _last)]
features_train = features_train[features_train.Signal!=0]
features_test = features_test[features_test.Signal!=0]



features_train['slow_K5'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.slow_K5.astype(np.float32)).reshape(-1, 1)))
features_train['slow_D5'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.slow_D5.astype(np.float32)).reshape(-1, 1)))
features_train['deltaHL'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.deltaHL.astype(np.float32)).reshape(-1, 1)))
features_train['deltaOC'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.deltaOC.astype(np.float32)).reshape(-1, 1)))
features_train['deltaHC'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.deltaHC.astype(np.float32)).reshape(-1, 1)))
features_train['deltaLC'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.deltaLC.astype(np.float32)).reshape(-1, 1)))
features_train['deltaOH'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.deltaOH.astype(np.float32)).reshape(-1, 1)))
features_train['deltaOL'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.deltaOL.astype(np.float32)).reshape(-1, 1)))


features_test['slow_K5'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.slow_K5.astype(np.float32)).reshape(-1, 1)))
features_test['slow_D5'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.slow_D5.astype(np.float32)).reshape(-1, 1)))
features_test['deltaHL'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.deltaHL.astype(np.float32)).reshape(-1, 1)))
features_test['deltaOC'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.deltaOC.astype(np.float32)).reshape(-1, 1)))
features_test['deltaHC'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.deltaHC.astype(np.float32)).reshape(-1, 1)))
features_test['deltaLC'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.deltaLC.astype(np.float32)).reshape(-1, 1)))
features_test['deltaOH'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.deltaOH.astype(np.float32)).reshape(-1, 1)))
features_test['deltaOL'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.deltaOL.astype(np.float32)).reshape(-1, 1)))



features_oos['slow_K5'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.slow_K5.astype(np.float32)).reshape(-1, 1)))
features_oos['slow_D5'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.slow_D5.astype(np.float32)).reshape(-1, 1)))
features_oos['deltaOC'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.deltaOC.astype(np.float32)).reshape(-1, 1)))
features_oos['deltaOC'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.deltaOC.astype(np.float32)).reshape(-1, 1)))
features_oos['deltaHC'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.deltaHC.astype(np.float32)).reshape(-1, 1)))
features_oos['deltaLC'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.deltaLC.astype(np.float32)).reshape(-1, 1)))
features_oos['deltaOH'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.deltaOH.astype(np.float32)).reshape(-1, 1)))
features_oos['deltaOL'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.deltaOL.astype(np.float32)).reshape(-1, 1)))


#_model = MLPClassifier(hidden_layer_sizes=(300,300), activation='logistic', solver='adam', alpha=0.000001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.15, max_iter=800, shuffle=True, random_state=26, tol=0.000001, verbose=1, warm_start=True, momentum=0.99, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
#

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier,RadiusNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

_model = DecisionTreeClassifier(criterion='entropy',random_state=26,splitter='best',ccp_alpha=0.00007,max_features=7,class_weight={0: 1, 1: 1000})
_model.fit(features_train.drop(['Date','Symbol','TRACKER','Valid','Signal'],axis=1),features_train.Valid)
yhat = _model.predict(features_test.drop(['Date','Symbol','TRACKER','Valid','Signal'],axis=1))
accu = round(accuracy_score(features_test.Valid, yhat) * 100,2)
prec = round(precision_score(features_test.Valid, yhat,pos_label=1) * 100,2)
recall = round(recall_score(features_test.Valid, yhat) * 100,2)
f1 = round(f1_score(features_test.Valid, yhat) * 100,2)


print('Signaux - Accuracy :' ,accu,'%')
print('Signaux - Precision :',prec,'%')
print('Signaux - Recall :', recall,'%')
print('Achat - F-measure: :' ,f1,'%')
print('\n')
print(classification_report(features_test.Valid, yhat, target_names=['Neg', 'Pos']))
conf_matrix = pd.DataFrame(columns=['Bonnes_Estimations','Mauvaises_Estimations'])
tt = confusion_matrix(features_test.Valid, yhat, labels=[1,0])    #_model.classes_)

conf_matrix.loc['vrais-réels'] = tt[0]
conf_matrix.loc['faux-réels'] = tt[1]

print(conf_matrix)
print()
print(col.Fore.BLUE,'Signaux pour',col.Fore.YELLOW,x,col.Style.RESET_ALL)
_tp = tt[0][0]
_fn = tt[0][1]
_prec = round((tt[0][0]/(tt[0][0]+tt[0][1]))*100,2)
_rec = round((tt[0][1]/(tt[0][0]+tt[0][1]))*100,2)

print('Vrais signaux trouvés    : ',tt[0][0])
print('Vrais signaux non trouvé :',tt[0][1])
print('Total des vrais signaux  :',tt[0][0]+tt[0][1])
if _prec > 69 and prec > 69 :
    print(col.Fore.GREEN)
elif _prec < 51 or prec < 51 :
    print(col.Fore.RED)
else:
    print(col.Fore.YELLOW)
print('Precision : ',_prec,'%')
if _rec > 69 and _rec > 69 :
    print(col.Fore.GREEN)
elif _rec < 51 or _rec < 51 :
    print(col.Fore.RED)
else:
    print(col.Fore.YELLOW)
print('Recall',_rec,'%')
print(col.Style.RESET_ALL)
joblib.dump(_model,'1')

Signaux - Accuracy : 12.16 %
Signaux - Precision : 7.64 %
Signaux - Recall : 98.25 %
Achat - F-measure: : 14.18 %


              precision    recall  f1-score   support

         Neg       0.97      0.05      0.10      4288
         Pos       0.08      0.98      0.14       342

    accuracy                           0.12      4630
   macro avg       0.53      0.52      0.12      4630
weighted avg       0.91      0.12      0.10      4630

            Bonnes_Estimations Mauvaises_Estimations
vrais-réels                336                     6
faux-réels                4061                   227

 Signaux pour  EUR/USD 
Vrais signaux trouvés    :  336
Vrais signaux non trouvé : 6
Total des vrais signaux  : 342

Precision :  98.25 %

Recall 1.75 %

CPU times: user 2.12 s, sys: 529 ms, total: 2.65 s
Wall time: 2.6 s


['1']

In [107]:
%%time
TICKER_LIST = ['EUR/USD']
x=TICKER_LIST[0]
df_all = joblib.load(x.replace('/','')+'_DF_ALL')
####### CHOIX DE LA TIME FRAME
_period = 'm5'
##### Récupération de la liste des tickers à partir du répertoire où sont rangées les bases
TICKER_LIST = ['EUR/USD']
TIK = ['JPY','CHF','CAD','GBP','AUD','NZD','SEK','NOK','MXN','ZAR','TRY','ILS','CNH','USD','HKD','0']
RATE = [105.4,0.8989,1.276,0.877,0.7676,0.7201,8.3684,8.5238,20.09,14.81,7.03,3.28,6.458,1,7.7527,1.21]
df_ratefx = pd.DataFrame(index=TIK)
df_ratefx['rate'] = RATE
#############################
##### F E A T U R I N G #####
#############################

print(df_all.shape)
# Make our choice for the split, compliant to our backtest 
_start = '2010-01-01' # start the train there '2010-01-01'
_mid = '2018-08-29' # stop the train and begin the test there '2016-08-31'
_stop = '2019-12-30' # stop the test there. After that, it is kept for oos
_last = '2021-02-15' # '2020-12-31'
#### S T R A T E G Y

df_all = stochastic(df_all)
##### On backtest selon le ticker selectionné sur la période déterminée

x=TICKER_LIST[0]
print(x)
_year_bottom = _stop
_year_top = _last
_nb_bougie_exit = 2000000
_trigger_reengage = 0
_trigger_target = 1
_trigger_invers = 0
_trigger_sl = 1
_verbose = 0
_cash_ini = 200000
_rate = 1/df_ratefx.loc[x[4:],'rate']
backtest = df_all[df_all.Symbol == x.replace('/','')]
_target =  0.002
_exposure = 10
_size = 200000
_sl =  0.001
_open_hour = 8 # day only
_close_hour = 23 # day only
_window = 0 # day only
print()
print(col.Fore.RED,'###############################################################################################')
print(' ####################################### OOS WITHOUT AI ########################################')
print(' ###############################################################################################',col.Style.RESET_ALL)
##### Backtest Over Night
TRACKER,_nb_looser = bt(backtest,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)

print()
print(col.Fore.CYAN,'###############################################################################################')
print(' #################################### TRAIN/TEST WITHOUT AI ####################################')
print(' ###############################################################################################',col.Style.RESET_ALL)
_year_bottom = _start
_year_top = _stop

TRACKER,_nb_looser = bt(backtest,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)

print()
print(col.Fore.BLUE,'###############################################################################################')
print(' #################################### DENOISING & ENHANCING ####################################')
print(' ###############################################################################################',col.Style.RESET_ALL)
while _nb_looser > 0 :
    
    df_all['TRACKER'] = np.where(df_all.index.isin(TRACKER),1,0)
    df_all['Valid'] = np.where(((df_all.Signal!=0)&(df_all.TRACKER==1)),1,0)
    df_all['Signal'] = np.where(((df_all.Valid==1)&(df_all.Signal==1)),1,np.where(((df_all.Valid==1)&(df_all.Signal==-1)),-1,0))
    backtest = df_all[df_all.Symbol == x.replace('/','')]
    ##### Purification of signal by denoising and enhancing
    TRACKER,_nb_looser = bt(backtest,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
            _rate,x,_target,_exposure,_size,_sl)

df_all = joblib.load(x.replace('/','')+'_DF_ALL')
df_all = stochastic(df_all)
#features = featuring(df_all)
features = df_all.copy()

features['TRACKER'] = np.where(features.index.isin(TRACKER),1,0)

# First, we must have an output. We'll call it 'Valid'. It wil be where Tracker & Signal are both to 1
features['Valid'] = np.where(((features.Signal!=0)&(features.TRACKER==1)),1,0) # Don't miss the point that even a Signal -1 must be considered as a good one by TRACKER

# Let's isolate the ticker on which we made the test
#features = features[features.Symbol==x.replace('/','')]
#features = df_all.copy()
# And drop the nan
features = features.dropna()
##### Signal is from strategy. This is potential good one. But we have to create the TRACKER column where the Signal where efficient

features['deltaHL'] = features['High'] - features['Low']
features['deltaOC'] = features['Open'] - features['Close']
features['deltaHC'] = features['High'] - features['Close']
features['deltaLC'] = features['Low'] - features['Close']
features['deltaOH'] = features['Open'] - features['High']
features['deltaOL'] = features['Open'] - features['Low']

features = features.drop(['OpenBid','OpenAsk','HighBid','LowBid','CloseBid','HighAsk','LowAsk','CloseAsk','Open','High','Low','Close'],axis=1)

features = features.dropna()
features = features[features.Symbol==_ticker]
features['TRACKER'] = np.where(features.index.isin(TRACKER),1,0)
features_train = features[(features.Date>=_start)&(features.Date<=_mid)]
features_test = features[(features.Date>_mid)&(features.Date<=_stop)]
features_oos = features[(features.Date>_stop)&(features.Date <= _last)]
features_train = features_train[features_train.Signal!=0]
features_test = features_test[features_test.Signal!=0]



features_train['slow_K5'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.slow_K5.astype(np.float32)).reshape(-1, 1)))
features_train['slow_D5'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.slow_D5.astype(np.float32)).reshape(-1, 1)))
features_train['deltaHL'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.deltaHL.astype(np.float32)).reshape(-1, 1)))
features_train['deltaOC'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.deltaOC.astype(np.float32)).reshape(-1, 1)))
features_train['deltaHC'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.deltaHC.astype(np.float32)).reshape(-1, 1)))
features_train['deltaLC'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.deltaLC.astype(np.float32)).reshape(-1, 1)))
features_train['deltaOH'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.deltaOH.astype(np.float32)).reshape(-1, 1)))
features_train['deltaOL'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_train.deltaOL.astype(np.float32)).reshape(-1, 1)))


features_test['slow_K5'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.slow_K5.astype(np.float32)).reshape(-1, 1)))
features_test['slow_D5'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.slow_D5.astype(np.float32)).reshape(-1, 1)))
features_test['deltaHL'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.deltaHL.astype(np.float32)).reshape(-1, 1)))
features_test['deltaOC'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.deltaOC.astype(np.float32)).reshape(-1, 1)))
features_test['deltaHC'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.deltaHC.astype(np.float32)).reshape(-1, 1)))
features_test['deltaLC'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.deltaLC.astype(np.float32)).reshape(-1, 1)))
features_test['deltaOH'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.deltaOH.astype(np.float32)).reshape(-1, 1)))
features_test['deltaOL'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_test.deltaOL.astype(np.float32)).reshape(-1, 1)))



features_oos['slow_K5'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.slow_K5.astype(np.float32)).reshape(-1, 1)))
features_oos['slow_D5'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.slow_D5.astype(np.float32)).reshape(-1, 1)))
features_oos['deltaOC'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.deltaOC.astype(np.float32)).reshape(-1, 1)))
features_oos['deltaOC'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.deltaOC.astype(np.float32)).reshape(-1, 1)))
features_oos['deltaHC'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.deltaHC.astype(np.float32)).reshape(-1, 1)))
features_oos['deltaLC'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.deltaLC.astype(np.float32)).reshape(-1, 1)))
features_oos['deltaOH'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.deltaOH.astype(np.float32)).reshape(-1, 1)))
features_oos['deltaOL'] = quantile_transform(scaler.fit_transform(np.nan_to_num(features_oos.deltaOL.astype(np.float32)).reshape(-1, 1)))


# Proceed an MaxAbsScaler on features
#features_train,features_test,features_oos = scaling(features,x.replace('/',''),TRACKER,_start,_mid,_stop,_last,scaler=MaxAbsScaler())
#features_train,features_test,features_oos = quantile(features_train,features_test,features_oos,quantile_transform)


#_model = MLPClassifier(hidden_layer_sizes=(300,15), activation='relu', solver='adam', alpha=0.000001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=400, shuffle=True, random_state=26, tol=0.0000001, verbose=False, warm_start=True, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.8, beta_2=0.999, epsilon=1e-08)
_model0 = joblib.load('0')
_model1 = joblib.load('1')


df_all_oos = df_all[(df_all.Date > _stop)&(df_all.Date <= _last)].dropna()

df_all_oos = df_all_oos[df_all_oos.Symbol==x.replace('/','')]


df_all_oos['Valid0'] = _model0.predict(features_oos.drop(['Date','Symbol','Signal','TRACKER','Valid'],axis=1))
df_all_oos['Valid1'] = _model0.predict(features_oos.drop(['Date','Symbol','Signal','TRACKER','Valid'],axis=1))


df_all_oos['Signal'] = np.where((df_all_oos.Signal==1)&((df_all_oos.Valid0==0)),1,np.where((df_all_oos.Signal==-1)&((df_all_oos.Valid0==0)),-1,0)) #&(df_all_oos.Valid1==1)
 
##### On backtest selon le ticker selectionné sur la période déterminée

joblib.dump(_model,'Save_'+x.replace('/','')+'_m5.sav')

print(x)
_year_bottom = _stop
_year_top = _last

#_trigger_invers = 1
#_target = 0.017
##### Backtest Over Night
FINAL_TRACKER = bt(df_all_oos,_year_bottom,_year_top,_nb_bougie_exit,_trigger_reengage,_trigger_target,_trigger_invers,_trigger_sl,_verbose,_cash_ini,\
        _rate,x,_target,_exposure,_size,_sl)

(835382, 16)
EUR/USD

 ###############################################################################################
 ####################################### OOS WITHOUT AI ########################################
 ############################################################################################### 
Librairies imported

Début des opérations horodatée à 2021-03-04 12:53:46.698980

Chargement de la nouvelle base


 Le rate du ticker EUR/USD est à  1.0 
Bases chargées
TETEL process effectué
 ENTERING THE BACKTEST 
100%|██████████| 84912/84912 [00:05<00:00, 15227.90it/s]
 For ticker  EUR/USD 
 
Total Profit & Loss : $  -50302.0 . En  1460   transactions.
 
Winners Number : 457 
 
Loosers number : 1003 
BT's execution time 0:00:11.916744
 EUR/USD  results 
 Tested Period 2019-12-30  à 2021-02-15 
 Total Number of trades 1460 
Started Cash : 200000
P&L in currency:  -50302.0$ 
P&L in %:  -25.15% 
Average trade duration 44.59
# Winners  457.0
# Winners long  210.0
# Winners short